Motivation: auc/gini with keras

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
print('Reading files')
train  =pd.read_csv("../input/train.csv")
test   =pd.read_csv("../input/test.csv")
col_x= train.columns.drop(['target'])
col  = train.columns.drop(['id','target'])
print('OK')

Reading files
OK


Blablah keras toys you want to use

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import keras.models 

Using TensorFlow backend.


Kaggle discussion/kernels metrics

In [4]:
import tensorflow as tf
import keras.backend as K

def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score

# FROM https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/41108
def jacek_auc(y_true, y_pred):
   score, up_opt = tf.metrics.auc(y_true, y_pred)
   #score, up_opt = tf.contrib.metrics.streaming_auc(y_pred, y_true)    
   K.get_session().run(tf.local_variables_initializer())
   with tf.control_dependencies([up_opt]):
       score = tf.identity(score)
   return score

# FROM https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/41015
# AUC for a binary classifier
def discussion41015_auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)

#---------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N

#----------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P

Any model, just an example:

In [5]:
def model_relu1():
    model = Sequential()
    model.add(Dense(1024, input_dim=57, activation='relu', name='in'))
    model.add(Dense(   1, activation='sigmoid', name='out'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=[jacek_auc,discussion41015_auc])
    return model


# Option 1 - magic , create an callback and handle everything

In [6]:
#go here, it's easier to understand callbacks reading keras source code:
#   https://github.com/fchollet/keras/blob/master/keras/callbacks.py#L838
#   https://github.com/fchollet/keras/blob/master/keras/engine/training.py#L1040

from sklearn.metrics import roc_auc_score
class GiniWithEarlyStopping(keras.callbacks.Callback):
    def __init__(self, min_delta=0, patience=0, verbose=0, predict_batch_size=1024):
        #print("self vars: ",vars(self))  #uncomment and discover some things =)
        
        # FROM EARLY STOP
        super(GiniWithEarlyStopping, self).__init__()
        self.patience = patience
        self.verbose = verbose
        self.min_delta = min_delta
        self.wait = 0
        self.stopped_epoch = 0
        self.monitor_op = np.greater
        self.predict_batch_size=predict_batch_size
    
    def on_batch_begin(self, batch, logs={}):
        if(self.verbose > 1):
            if(batch!=0):
                print("")
            print("Hi! on_batch_begin() , batch=",batch,",logs:",logs)
            #print("self vars: ",vars(self))  #uncomment and discover some things =)
    
    def on_batch_end(self, batch, logs={}):
        if(self.verbose > 1):
            print("Hi! on_batch_end() , batch=",batch,",logs:",logs)
            #print("self vars: ",vars(self))  #uncomment and discover some things =)
    
    def on_train_begin(self, logs={}):
        if(self.verbose > 1):
            print("Hi! on_train_begin() ,logs:",logs)
            #print("self vars: ",vars(self))  #uncomment and discover some things =)

        # FROM EARLY STOP
        # Allow instances to be re-used
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
    
    def on_train_end(self, logs={}):
        if(self.verbose > 1):
            print("Hi! on_train_end() ,logs:",logs)
            #print("self vars: ",vars(self))  #uncomment and discover some things =)

        # FROM EARLY STOP
        if self.stopped_epoch > 0 and self.verbose > 0:
            print('Epoch ',self.stopped_epoch,': GiniEarlyStopping')
    
    def on_epoch_begin(self, epoch, logs={}):
        if(self.verbose > 1):
            print("Hi! on_epoch_begin() , epoch=",epoch,",logs:",logs)
            #print("self vars: ",vars(self))  #uncomment and discover some things =)

    def on_epoch_end(self, epoch, logs={}):
        if(self.validation_data):
            y_hat_val=self.model.predict(self.validation_data[0],batch_size=self.predict_batch_size)
            
        if(self.verbose > 1):
            print("Hi! on_epoch_end() , epoch=",epoch,",logs:",logs)
            #print("self vars: ",vars(self))  #uncomment and discover some things =)
        
        #i didn't found train data to check gini on train set (@TODO HERE)
        # from source code of Keras: https://github.com/fchollet/keras/blob/master/keras/engine/training.py#L1127
        # for cbk in callbacks:
        #     cbk.validation_data = val_ins
        # Probably we will need to change keras... 
        # 
        
            print("    GINI Callback:")
            if(self.validation_data):
                print('        validation_data.inputs       : ',np.shape(self.validation_data[0]))
                print('        validation_data.targets      : ',np.shape(self.validation_data[1]))
                print("        roc_auc_score(y_real,y_hat)  : ",roc_auc_score(self.validation_data[1], y_hat_val ))
                print("        gini_normalized(y_real,y_hat): ",gini_normalized(self.validation_data[1], y_hat_val))
                print("        roc_auc_scores*2-1           : ",roc_auc_score(self.validation_data[1], y_hat_val)*2-1)
        
            print('    Logs (others metrics):',logs)
        # FROM EARLY STOP
        if(self.validation_data):
            if (self.verbose == 1):
                print("\n GINI Callback:",gini_normalized(self.validation_data[1], y_hat_val))
            current = gini_normalized(self.validation_data[1], y_hat_val)
            
            # we can include an "gambiarra" (very usefull brazilian portuguese word)
            # to logs (scores) and use others callbacks too....
            # logs['gini_val']=current
            
            if self.monitor_op(current - self.min_delta, self.best):
                self.best = current
                self.wait = 0
            else:
                self.wait += 1
                if self.wait >= self.patience:
                    self.stopped_epoch = epoch
                    self.model.stop_training = True


In [7]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# reduce train size, just to this kernel example
t=train[0:1000]
# batch_size=500 ~= 2 batchs
estimator = KerasClassifier(build_fn=model_relu1, nb_epoch=3, batch_size=500, verbose=1)



cb = [
    # verbose =2 make many prints (nice to learn keras callback)
    GiniWithEarlyStopping(patience=1, verbose=2) 
]

estimator.fit(t[col].values,t['target'],epochs=100,validation_split=.2,callbacks=cb)




Train on 800 samples, validate on 200 samples
Hi! on_train_begin() ,logs: {}
Hi! on_epoch_begin() , epoch= 0 ,logs: {}
Epoch 1/100
Hi! on_batch_begin() , batch= 0 ,logs: {'batch': 0, 'size': 500}
Hi! on_batch_end() , batch= 0 ,logs: {'batch': 0, 'size': 500, 'loss': 0.46887365, 'jacek_auc': 0.0, 'discussion41015_auc': 0.4920834}
500/800 [=================>............] - ETA: 1s - loss: 0.4689 - jacek_auc: 0.0000e+00 - discussion41015_auc: 0.4921
Hi! on_batch_begin() , batch= 1 ,logs: {'batch': 1, 'size': 300}
Hi! on_batch_end() , batch= 1 ,logs: {'batch': 1, 'size': 300, 'loss': 0.21426727, 'jacek_auc': 0.4925521, 'discussion41015_auc': 0.45023149}
Hi! on_epoch_end() , epoch= 0 ,logs: {'val_loss': 0.14205174148082733, 'val_jacek_auc': 0.48828127980232239, 'val_discussion41015_auc': 0.36224490404129028, 'loss': 0.3733962569385767, 'jacek_auc': 0.18470703810453415, 'discussion41015_auc': 0.47638893499970436}
    GINI Callback:
        validation_data.inputs       :  (200, 57)
        va

I don't know why the last line " < keras.callbacks.History at 0x..... > " anyone please check it and comment to fix

In [8]:
cb = [
    # verbose =1 print gini per epoch
    GiniWithEarlyStopping(patience=1, verbose=1) 
]

estimator.fit(t[col].values,t['target'],epochs=100,validation_split=.2,callbacks=cb)



Train on 800 samples, validate on 200 samples
Epoch 1/100
500/800 [=================>............] - ETA: 1s - loss: 0.2019 - jacek_auc: 0.0000e+00 - discussion41015_auc: 0.5649
 GINI Callback: 0.0892857142857
800/800 [==============================] - 5s - loss: 0.2309 - jacek_auc: 0.2132 - discussion41015_auc: 0.4864 - val_loss: 0.1062 - val_jacek_auc: 0.5061 - val_discussion41015_auc: 0.5255
Epoch 2/100
500/800 [=================>............] - ETA: 0s - loss: 0.1959 - jacek_auc: 0.5178 - discussion41015_auc: 0.4669
 GINI Callback: 0.0612244897959
800/800 [==============================] - 0s - loss: 0.1883 - jacek_auc: 0.5106 - discussion41015_auc: 0.5392 - val_loss: 0.1664 - val_jacek_auc: 0.5234 - val_discussion41015_auc: 0.5281
Epoch  1 : GiniEarlyStopping


I don't know why the last line " < keras.callbacks.History at 0x..... > " anyone please check it and comment to fix

In [9]:
cb = [
    # verbose =0 don't print
    GiniWithEarlyStopping(patience=1, verbose=0) 
]

estimator.fit(t[col].values,t['target'],epochs=100,validation_split=.2,callbacks=cb)



Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 5s - loss: 0.2735 - jacek_auc: 0.1710 - discussion41015_auc: 0.4796 - val_loss: 0.1443 - val_jacek_auc: 0.5124 - val_discussion41015_auc: 0.4885
Epoch 2/100
800/800 [==============================] - 0s - loss: 0.2955 - jacek_auc: 0.5374 - discussion41015_auc: 0.3256 - val_loss: 0.1301 - val_jacek_auc: 0.5104 - val_discussion41015_auc: 0.4936
Epoch 3/100
800/800 [==============================] - 0s - loss: 0.2370 - jacek_auc: 0.5146 - discussion41015_auc: 0.4312 - val_loss: 0.1250 - val_jacek_auc: 0.5144 - val_discussion41015_auc: 0.5153


# Option 2 - magic, Include metric in logs dictionary

example with Roc

In [10]:
from sklearn.metrics import roc_auc_score
class RocAucMetricCallback(keras.callbacks.Callback):
    def __init__(self, predict_batch_size=1024, include_on_batch=False):
        super(RocAucMetricCallback, self).__init__()
        self.predict_batch_size=predict_batch_size
        self.include_on_batch=include_on_batch

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        if(self.include_on_batch):
            logs['roc_auc_val']=float('-inf')
            if(self.validation_data):
                logs['roc_auc_val']=roc_auc_score(self.validation_data[1], 
                                                  self.model.predict(self.validation_data[0],
                                                                     batch_size=self.predict_batch_size))

    def on_train_begin(self, logs={}):
        if not ('roc_auc_val' in self.params['metrics']):
            self.params['metrics'].append('roc_auc_val')

    def on_train_end(self, logs={}):
        pass

    def on_epoch_begin(self, epoch, logs={}):
        pass

    def on_epoch_end(self, epoch, logs={}):
        logs['roc_auc_val']=float('-inf')
        if(self.validation_data):
            logs['roc_auc_val']=roc_auc_score(self.validation_data[1], 
                                              self.model.predict(self.validation_data[0],
                                                                 batch_size=self.predict_batch_size))



In [11]:
from keras.callbacks import EarlyStopping
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# reduce train size, just to this kernel example
t=train[0:1000]
# batch_size=500 ~= 2 batchs
estimator = KerasClassifier(build_fn=model_relu1, nb_epoch=3, batch_size=500, verbose=1)

cb = [
    RocAucMetricCallback(), # include it before EarlyStopping!
    EarlyStopping(monitor='roc_auc_val',patience=1, verbose=2) 
]

estimator.fit(t[col].values,t['target'],epochs=100,validation_split=.2,callbacks=cb)




Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 7s - loss: 0.3734 - jacek_auc: 0.1847 - discussion41015_auc: 0.4764 - val_loss: 0.1420 - val_jacek_auc: 0.4883 - val_discussion41015_auc: 0.3622 - roc_auc_val: 0.4834
Epoch 2/100
800/800 [==============================] - 0s - loss: 0.2979 - jacek_auc: 0.5192 - discussion41015_auc: 0.3201 - val_loss: 0.1865 - val_jacek_auc: 0.5101 - val_discussion41015_auc: 0.2334 - roc_auc_val: 0.4962
Epoch 3/100
800/800 [==============================] - 0s - loss: 0.3850 - jacek_auc: 0.5232 - discussion41015_auc: 0.1627 - val_loss: 0.1970 - val_jacek_auc: 0.5104 - val_discussion41015_auc: 0.2321 - roc_auc_val: 0.4936
Epoch 00002: early stopping


"Epoch 00002: early stopping" - Nice =D


In [12]:
cb = [
    EarlyStopping(monitor='roc_auc_val',patience=1, verbose=2), 
    RocAucMetricCallback(), # include it before EarlyStopping! i told you...
]
estimator.fit(t[col].values,t['target'],epochs=100,validation_split=.2,callbacks=cb)


Train on 800 samples, validate on 200 samples
Epoch 1/100
500/800 [=================>............] - ETA: 2s - loss: 0.2019 - jacek_auc: 0.0000e+00 - discussion41015_auc: 0.5649

/opt/conda/lib/python3.6/site-packages/Keras-2.0.6-py3.6.egg/keras/callbacks.py:496: RuntimeWarning: Early stopping conditioned on metric `roc_auc_val` which is not available. Available metrics are: val_loss,val_jacek_auc,val_discussion41015_auc,loss,jacek_auc,discussion41015_auc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'